In [1]:
import datetime as dt

import matplotlib.pyplot as plt
import mplfinance as mpf
import numpy as np
import pandas as pd
import pandas_market_calendars as mcal
import plotly.graph_objects as go
import polars as pl
from dash import Dash, dcc, html
from plotly.subplots import make_subplots

nse = mcal.get_calendar("NSE")

pd.set_option("display.max_rows", 25_000)
pd.set_option("display.max_columns", 500)
pl.Config.set_tbl_cols(500)
pl.Config.set_tbl_rows(10_000)
pd.options.display.float_format = "{:.4f}".format

import sys

sys.path.append("..")
from tooling.enums import AssetClass, Index, Spot, StrikeSpread
from tooling.fetch import fetch_option_data, fetch_spot_data
from tooling.filter import find_atm, option_tool

from fetching_from_local_db.enums import AssetClass, Index, StrikeSpread
from fetching_from_local_db.fetch_from_db import _fetch_batch, fetch_data, fetch_spot_data

In [2]:
bnf = pd.read_csv('WAVE_1x_bnf.csv')
nifty = pd.read_csv('WAVE_1x_nifty.csv')
# fnf = pd.read_csv('MACD_fin_last.csv')
midcp = pd.read_csv('WAVE_1x_midcp.csv')
# bankex = pd.read_csv('MACD_bankex.csv')
sensex = pd.read_csv('WAVE_1x_sensex.csv')

bnf['Index'] = 'BANKNIFTY'
nifty['Index'] = 'NIFTY'
# fnf['Index'] = 'FINNIFTY'
midcp['Index'] = 'MIDCPNIFTY'
# bankex['Index'] = 'BANKEX'
sensex['Index'] = 'SENSEX'

In [3]:
# combined_df = pd.concat([bnf, nifty, fnf, midcp, bankex, sensex], ignore_index=True)
combined_df = pd.concat([bnf, nifty, midcp, sensex], ignore_index=True)

In [4]:
combined_df.tail(30)

,Unnamed: 0,Signal Generated At,Trade Type,Entry Time,Entry Price,Initial SL,Final SL,Exit Time,Exit Price,Points Captured,After Costs,PnL,Remarks,Qty,Leverage,ROI%,Trade Year,Trade Month,Index
1766,306,2024-03-01 09:15:00,LONG,2024-03-01 10:15:00,73215.4900,72776.1971,73446.4358,2024-03-11 13:15:00,73683.9700,468.4800,453.7901,4537.9005,MACD Reversal,10,1,0.4538,2024,3,SENSEX
1767,307,2024-03-21 12:15:00,LONG,2024-03-21 13:15:00,72672.4500,72236.4153,72236.4153,2024-03-22 09:15:00,72227.8300,-444.6200,-459.1100,-4591.1003,Gap Open Outside ISL,10,1,-0.4591,2024,3,SENSEX
1768,308,2024-03-22 09:15:00,LONG,2024-03-22 10:15:00,72722.0300,72285.6978,72684.5347,2024-03-26 09:15:00,72536.4600,-185.5700,-200.0958,-2000.9585,TSL Hit,10,1,-0.2001,2024,3,SENSEX
1769,309,2024-03-26 11:15:00,LONG,2024-03-26 12:15:00,72648.9700,72213.0762,72213.0762,2024-03-26 14:15:00,72447.7800,-201.1900,-215.6997,-2156.9968,MACD Reversal,10,1,-0.2157,2024,3,SENSEX
1770,310,2024-03-27 09:15:00,LONG,2024-03-27 10:15:00,73009.3500,72571.2939,74351.5315,2024-04-12 13:15:00,74326.5700,1317.2200,1302.4864,13024.8641,TSL Hit,10,1,1.3025,2024,3,SENSEX
1771,311,2024-04-22 12:15:00,LONG,2024-04-22 13:15:00,73503.5600,73062.5386,74169.3767,2024-05-03 11:15:00,73935.1800,431.6200,416.8761,4168.7613,TSL Hit,10,1,0.4169,2024,4,SENSEX
1772,312,2024-05-14 12:15:00,LONG,2024-05-14 13:15:00,73165.1400,72726.1492,72726.1492,2024-05-16 11:15:00,72726.1492,-438.9908,-453.5800,-4535.7997,Initial SL Hit,10,1,-0.4536,2024,5,SENSEX
1773,313,2024-05-16 11:15:00,LONG,2024-05-16 12:15:00,73301.4600,72861.6512,72861.6512,2024-05-16 12:15:00,72861.6512,-439.8088,-454.4251,-4544.2507,Initial SL Hit,10,1,-0.4544,2024,5,SENSEX
1774,314,2024-05-16 14:15:00,LONG,2024-05-16 15:15:00,73671.1000,73229.0734,74535.8958,2024-05-29 09:15:00,74735.6200,1064.5200,1049.6793,10496.7933,MACD Reversal,10,1,1.0497,2024,5,SENSEX
1775,315,2024-06-03 10:15:00,LONG,2024-06-03 11:15:00,76145.7000,75688.8258,75688.8258,2024-06-04 09:15:00,75688.8258,-456.8742,-472.0577,-4720.5765,Initial SL hit,10,1,-0.4721,2024,6,SENSEX


In [6]:
combined_df.drop(columns=['Unnamed: 0'], inplace=True)

In [7]:
combined_df['Signal Generated At'] = pd.to_datetime(combined_df['Signal Generated At'])
combined_df = combined_df.sort_values(by='Signal Generated At')

In [8]:
# ONLY NSE
combined_df['Portfolio'] = 1000000
combined_df.loc[(combined_df['Trade Year'] <= 2017), 'Allocation'] = 500000
combined_df.loc[(combined_df['Trade Year'] > 2017) & (combined_df['Trade Year'] <= 2021), 'Allocation'] = 333333
combined_df.loc[(combined_df['Trade Year'] >= 2022), 'Allocation'] = 250000

# NSE + BSE
# combined_df['Portfolio'] = 60000000
# combined_df.loc[(combined_df['Trade Year'] <= 2017), 'Allocation'] = 12000000
# combined_df.loc[(combined_df['Trade Year'] > 2017) & (combined_df['Trade Year'] < 2022), 'Allocation'] = 12000000
# combined_df.loc[(combined_df['Trade Year'] >= 2022), 'Allocation'] = 10000000

In [9]:
combined_df['Qty apAlloc'] = combined_df['Qty'] * (combined_df['Allocation'] / 1000000)
# combined_df['Qty apAlloc'] = combined_df['Qty']

In [10]:
combined_df['PnL apAlloc'] = combined_df['After Costs'] * combined_df['Qty apAlloc']

In [11]:
combined_df['ROI% PF'] = combined_df['PnL apAlloc'] * 100 / combined_df['Portfolio']

In [12]:
# combined_df['ROI% PF'] = combined_df['PnL'] * 100 / combined_df['Portfolio']

In [13]:
# combined_df['ROI% PF'] /= 5

In [18]:
def generate_stats(tb_expiry):
    stats_df8 = pd.DataFrame(
        index=range(2017, 2025),
        columns=[
            "Total ROI",
            "Total Trades",
            "Win Rate",
            "Avg Profit% per Trade",
            "Avg Loss% per Trade",
            
            "Max Drawdown",
            "ROI/DD Ratio",
            
        ],
    )
    combined_df_sorted = tb_expiry
    # combined_df_sorted = tb_expiry_ce
    # combined_df_sorted = tb_expiry_pe
    
    # Iterate over each year
    for year in range(2017, 2025):
        # Filter trades for the current year
        year_trades = combined_df_sorted[(combined_df_sorted["Trade Year"] == year)]
    
        # Calculate total ROI
        total_roi = year_trades["ROI% PF"].sum()
    
        # Calculate total number of trades
        total_trades = len(year_trades)
    
        # Calculate win rate
        win_rate = (year_trades["ROI% PF"] > 0).mean() * 100
    
        # Calculate average profit per trade
        avg_profit = year_trades[year_trades["ROI% PF"] > 0]["ROI% PF"].mean()
    
        # Calculate average loss per trade
        avg_loss = year_trades[year_trades["ROI% PF"] < 0]["ROI% PF"].mean()
    
        # Calculate maximum drawdown
        max_drawdown = (
            year_trades["ROI% PF"].cumsum() - year_trades["ROI% PF"].cumsum().cummax()
        ).min()
    
        # Calculate ROI/DD ratio
        roi_dd_ratio = total_roi / abs(max_drawdown)

        # variation = f'{signal_ma} , {trailing_ma}, {time_of_day}'
    
        # Store the statistics in the DataFrame
        stats_df8.loc[year] = [
            total_roi,
            total_trades,
            win_rate,
            avg_profit,
            avg_loss,
            max_drawdown,
            roi_dd_ratio,
            # variation,
        ]
    
    # Calculate overall statistics
    overall_total_roi = stats_df8["Total ROI"].sum()
    overall_total_trades = stats_df8["Total Trades"].sum()
    overall_win_rate = (combined_df_sorted["ROI% PF"] > 0).mean() * 100
    overall_avg_profit = combined_df_sorted[combined_df_sorted["ROI% PF"] > 0]["ROI% PF"].mean()
    overall_avg_loss = combined_df_sorted[combined_df_sorted["ROI% PF"] < 0]["ROI% PF"].mean()
    overall_max_drawdown = (
        combined_df_sorted["ROI% PF"].cumsum() - combined_df_sorted["ROI% PF"].cumsum().cummax()
    ).min()
    overall_roi_dd_ratio = overall_total_roi / abs(overall_max_drawdown)
    # overall_variation = variation
    
    # Store the overall statistics in the DataFrame
    stats_df8.loc["Overall"] = [
        overall_total_roi,
        overall_total_trades,
        overall_win_rate,
        overall_avg_profit,
        overall_avg_loss,
        overall_max_drawdown,
        overall_roi_dd_ratio,
        # overall_variation,
    ]
    return pd.DataFrame(stats_df8)

In [19]:
stats = generate_stats(combined_df)
stats

,Total ROI,Total Trades,Win Rate,Avg Profit% per Trade,Avg Loss% per Trade,Max Drawdown,ROI/DD Ratio
2017,8.1043,104,34.6154,0.5771,-0.1863,-1.9860,4.0806
2018,13.2275,201,38.3085,0.4313,-0.1612,-5.4068,2.4465
2019,11.4652,231,32.9004,0.4153,-0.1297,-5.0234,2.2824
2020,43.2141,216,37.5000,0.9213,-0.2327,-3.5415,12.2023
2021,21.9185,226,38.4956,0.5201,-0.1678,-5.7237,3.8294
2022,15.5075,275,36.0000,0.4276,-0.1524,-4.9352,3.1422
2023,16.0282,287,36.2369,0.3265,-0.0980,-2.7004,5.9354
2024,8.5609,256,40.2344,0.2531,-0.1145,-4.1547,2.0605
Overall,138.0263,1796.0000,36.9154,0.4642,-0.1498,-5.7237,24.1149


# WAVE - 1x Leverage - BNF, NF, SENSEX, MIDCP

In [14]:
stats = generate_stats(combined_df)
stats

,Total ROI,Total Trades,Win Rate,Avg Profit% per Trade,Avg Loss% per Trade,Max Drawdown,ROI/DD Ratio
2017,60.0005,104,34.6154,4.3811,-1.4370,-14.7981,4.0546
2018,84.7780,201,38.3085,2.5725,-0.9137,-32.1141,2.6399
2019,72.7492,231,32.9004,2.6533,-0.8316,-31.4005,2.3168
2020,275.6193,216,37.5000,5.7740,-1.4228,-23.5254,11.7158
2021,143.0377,226,38.4956,3.1303,-0.9302,-35.7072,4.0058
2022,102.8687,275,36.0000,2.7071,-0.9382,-31.8108,3.2338
2023,104.6306,287,36.2369,2.0743,-0.6071,-18.3720,5.6951
2024,67.1801,278,39.2086,1.8550,-0.7989,-27.7546,2.4205
Overall,910.8641,1818.0000,36.6558,2.9487,-0.9319,-35.7072,25.5092


# MACD Stats for -> BNF, NF, SENSEX, MIDCP

In [21]:
combined_df.to_csv('Wave-1xLev.csv', index=False)

In [20]:
combined_df.tail()

,Signal Generated At,Trade Type,Entry Time,Entry Price,Initial SL,Final SL,Exit Time,Exit Price,Points Captured,After Costs,PnL,Remarks,Qty,Leverage,ROI%,Trade Year,Trade Month,Index,Portfolio,Allocation,Qty apAlloc,PnL apAlloc,ROI% PF
894,2024-11-26 10:45:00,LONG,2024-11-26 11:15:00,52147.9500,51365.7308,52147.2100,2024-11-27 09:15:00,52078.1000,-69.8500,-80.2726,-1204.0891,TSL Hit,15,1,-0.1204,2024,11,BANKNIFTY,1000000,250000.0000,3.7500,-301.0223,-0.0301
895,2024-11-27 10:15:00,LONG,2024-11-27 10:45:00,52146.9500,51364.7458,52317.0400,2024-11-28 10:15:00,52239.1000,92.1500,81.7114,1225.6709,TSL Hit,15,1,0.1226,2024,11,BANKNIFTY,1000000,250000.0000,3.7500,306.4177,0.0306
896,2024-11-28 11:15:00,LONG,2024-11-28 11:45:00,52183.6000,51400.8460,51400.8460,2024-11-28 12:15:00,52085.9000,-97.7000,-108.1269,-1621.9042,MACD Reversal,15,1,-0.1622,2024,11,BANKNIFTY,1000000,250000.0000,3.7500,-405.4761,-0.0405
1795,2024-11-29 12:15:00,LONG,2024-11-29 13:15:00,79788.7500,79310.0175,79310.0175,2024-12-02 09:15:00,79310.0175,-478.7325,-494.6424,-4946.4238,Initial SL hit,10,1,-0.4946,2024,11,SENSEX,1000000,250000.0000,2.5000,-1236.6059,-0.1237
897,2024-12-03 09:15:00,LONG,2024-12-03 09:45:00,52487.2500,51699.9413,53142.2167,2024-12-05 09:45:00,53032.6000,545.3500,534.7980,8021.9702,TSL Hit,15,1,0.8022,2024,12,BANKNIFTY,1000000,250000.0000,3.7500,2005.4926,0.2005
